In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the path to your dataset containing fire and non-fire images
train_data_dir = 'E:/Fire Detection/Fire Dataset/data/img_data/train'
test_data_dir = 'E:/Fire Detection/Fire Dataset/data/img_data/train'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32


# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    subset='validation'
)

# Create a pre-trained MobileNetV2 model as the base
base_model = MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(img_width, img_height, 3)
)

# Add custom layers for classification
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save the model for future use
model.save('fire_detection_model.xml')


Found 1160 images belonging to 3 classes.
Found 288 images belonging to 3 classes.
Epoch 1/10
37/37 [==============================] - 452s 11s/step - loss: 0.7067 - accuracy: 0.6118 - val_loss: 0.6788 - val_accuracy: 0.6597
Epoch 2/10
37/37 [==============================] - 349s 9s/step - loss: 0.6591 - accuracy: 0.6483 - val_loss: 0.7438 - val_accuracy: 0.6667
Epoch 3/10
37/37 [==============================] - 368s 10s/step - loss: 0.6475 - accuracy: 0.6652 - val_loss: 0.7590 - val_accuracy: 0.6667
Epoch 4/10
37/37 [==============================] - 313s 8s/step - loss: 0.6436 - accuracy: 0.6652 - val_loss: 0.7691 - val_accuracy: 0.6667
Epoch 5/10
37/37 [==============================] - 281s 8s/step - loss: 0.6422 - accuracy: 0.6667 - val_loss: 0.7731 - val_accuracy: 0.6667
Epoch 6/10
37/37 [==============================] - 350s 9s/step - loss: 0.6413 - accuracy: 0.6667 - val_loss: 0.7726 - val_accuracy: 0.6667
Epoch 7/10
37/37 [==============================] - 299s 8s/step - lo

INFO:tensorflow:Assets written to: fire_detection_model.xml\assets


In [1]:
import cv2         # Library for openCV
import threading   # Library for threading -- which allows code to run in backend
import pygame   # Library for alarm sound
import smtplib     # Library for email sending
import urllib.request
import urllib.error
from email.message import EmailMessage
import ssl

fire_cascade = cv2.CascadeClassifier('fire_detection_cascade_model.xml') # To access xml file which includes positive and negative images of fire. (Trained images)

vid = cv2.VideoCapture(0) # To start camera this command is used "0" for laptop inbuilt camera and "1" for USB attahed camera for pc
runOnce = False # created boolean

# define rez
resTrack = (640,480) #or whatever
resWrite = (1280, 720)
vid.set( cv2.CAP_PROP_FRAME_WIDTH, resWrite[0])
vid.set( cv2.CAP_PROP_FRAME_HEIGHT, resWrite[1])

def play_alarm_sound_function():
    pygame.init()  # Initialize the video system (even though we're working with audio)
    pygame.mixer.init()
    pygame.mixer.music.load('E:/Fire Detection/Alarm Sound.mp3')
    pygame.mixer.music.play()
    pygame.event.wait()


def send_mail_function(): # defined function to send mail post fire detection using threading

    email_sender = 'kolidarshan123456@gmail.com'
    email_password = 'tpsc oeno yzhh uogh'
    email_reciver = 'rhlrtd2021@gmail.com'

    subject = 'Fire is detected'
    body = '"Warning: Fire accident has been detected. Please take necessary actions."'


    em = EmailMessage()

    em['from'] = email_sender
    em['to'] = email_reciver
    em['subject'] = subject
    em.set_content(body)

    content = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=content) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_reciver, em.as_string())


def internet_on():
    try:
        response = urllib.request.urlopen('https://www.google.co.in', timeout=1)
        return True
    except urllib.error.URLError as err:
        pass
    return False

		
while(True):
    Alarm_Status = False
    ret, frame = vid.read() # Value in ret is True # To read video frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # To convert frame into gray color
    fire = fire_cascade.detectMultiScale(frame, 1.2, 5) # Haar Cascade Classifier

    ## to highlight fire with square 
    # Inside the loop where fire is detected
    for (x, y, w, h) in fire:
        cv2.rectangle(frame, (x - 20, y - 20), (x + w + 20, y + h + 20), (0, 0, 255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Calculate confidence based on average pixel intensity
        confidence = roi_gray.mean()

        cv2.putText(frame, f'Fire: {confidence:.2f}', (x - 20, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        print("Fire alarm initiated")
        threading.Thread(target=play_alarm_sound_function).start()  # To call alarm thread

        if runOnce == False:
            print("Mail send initiated")
            threading.Thread(target=send_mail_function).start()  # To call alarm thread
            runOnce = True  
            
        if runOnce == True:
            print("Mail is already sent once")
            print(internet_on())
            runOnce = True

    cv2.imshow('Fire Detection System', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html
Fire alarm initiated
Mail send initiated
Mail is already sent once
False
Fire alarm initiated
Mail is already sent once


Exception in thread Thread-6 (send_mail_function):
Traceback (most recent call last):
  File "c:\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\kolid\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\kolid\AppData\Local\Temp\ipykernel_11252\3410820602.py", line 48, in send_mail_function
  File "c:\Python312\Lib\smtplib.py", line 1022, in __init__
    SMTP.__init__(self, host, port, local_hostname, timeout,
  File "c:\Python312\Lib\smtplib.py", line 255, in __init__
    (code, msg) = self.connect(host, port)
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\smtplib.py", line 341, in connect
    self.sock = self._get_socket(host, port, self.timeout)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Li

False
Fire alarm initiated
Mail is already sent once
False
Fire alarm initiated
Mail is already sent once


TimeoutError: The read operation timed out

Restarted Python 3.11.5